<div style="border-left: 5px solid green; padding: 10px; background-color:rgb(224, 247, 206);">
    <b>⚡️ Information:</b>
    Yihan's customized unet experiment
</div>

### Step 0 - Set & Check available memory

In [5]:
import tensorflow as tf
MEMORY_SIZE = 18
memory_limit = MEMORY_SIZE * 1024

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memory_limit)]  # Set limit to 15GB
        )
        print(f"Limited GPU memory usage to {MEMORY_SIZE}GB")
    except RuntimeError as e:
        print(e)

Limited GPU memory usage to 18GB


In [6]:

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for i, gpu in enumerate(gpus):
        device_name = f"GPU:{i}"  # Properly format the device name
        details = tf.config.experimental.get_memory_info(device_name)
        print(f"GPU: {device_name}")
        print(f"Current Memory Usage: {details['current']} bytes")
        print(f"Peak Memory Usage: {details['peak']} bytes")
else:
    print("No GPU devices found.")

for gpu in gpus:
    print(f"Memory growth for {gpu}: {tf.config.experimental.get_memory_growth(gpu)}")

GPU: GPU:0
Current Memory Usage: 0 bytes
Peak Memory Usage: 0 bytes
Memory growth for PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'): None


### Step 1 - Set up environment

In [7]:
import nibabel as nib
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from skimage.transform import rotate
from skimage.util import montage
import os
from sklearn.model_selection import train_test_split
import keras
import cv2
import tensorflow
import random
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import numpy as np
from keras.callbacks import CSVLogger
import keras.backend as K
import zipfile
import pandas as pd

#### paths, VOLUME_*, IMG_SIZE, N_CHANNELS

In [8]:
# Specify a sample path (here we will take the first patient of the Training dataset)
brats_index = 'BraTS-GLI-00077-000'
brats_sample_path = f'../brain_tumor_seg/data/raw_data/GLI_train/{brats_index}/{brats_index}-'

# Specify the root data path of training data
data_path_GLI_train_dir = f'../brain_tumor_seg/data/raw_data/GLI_train'


# Define selected slices range
VOLUME_START_AT = 40 
VOLUME_SLICES_PLUS = 76 
TUMOR_RATIO_LOWER_BOUND = 0.1
TUMOR_RATIO_UPPER_BOUND = 0.15

# For DataGenerator
IMG_SIZE = 256
N_CHANNELS = 2

### Not a Step - Explore data

In [ ]:
# Load the 4 MRI modalities and the segmentation located in the patient's path using the nibabel library
t1_img_sample=nib.load(brats_sample_path + 't1n.nii.gz')
t1ce_img_sample=nib.load(brats_sample_path + 't1c.nii.gz')
t2_img_sample=nib.load(brats_sample_path + 't2w.nii.gz')
flair_img_sample=nib.load(brats_sample_path + 't2f.nii.gz')
seg_img_sample=nib.load(brats_sample_path + 'seg.nii.gz')

# Get the image data
t1_data_sample = t1_img_sample.get_fdata()
t1ce_data_sample = t1ce_img_sample.get_fdata()
t2_data_sample = t2_img_sample.get_fdata()
flair_data_sample = flair_img_sample.get_fdata()
seg_data_sample = seg_img_sample.get_fdata()

# Plot the 100th slice of the 4 RMI modalities and the segmentation
slice_nb = 100

fig, axs = plt.subplots(1, 5, figsize=(20,20))
axs[0].imshow(t1_data_sample[:,:,slice_nb], cmap="gray")
axs[0].set_title('T1')
axs[1].imshow(t1ce_data_sample[:,:,slice_nb], cmap="gray")
axs[1].set_title('T1CE')
axs[2].imshow(t2_data_sample[:,:,slice_nb], cmap="gray")
axs[2].set_title('T2')
axs[3].imshow(flair_data_sample[:,:,slice_nb], cmap="gray")
axs[3].set_title('FLAIR')
axs[4].imshow(seg_data_sample[:,:,slice_nb], cmap="gray")
axs[4].set_title('Segmentation')
plt.savefig(f'./plt/seg_classes_{brats_index}.png', dpi=300)
plt.show()

In [ ]:
# Plot a RMI modality through all planes
slice_nb = 100

fig, axs2 = plt.subplots(1, 3, figsize=(10,10))

# Apply a 90° rotation with an automatic resizing, otherwise the display is less obvious to analyze
axs2[0].imshow(rotate(t1_data_sample[slice_nb,:,:], 90, resize=True), cmap="gray")
axs2[0].set_title('T1 - Sagittal View')

axs2[1].imshow(rotate(t1_data_sample[:,slice_nb,:], 90, resize=True), cmap="gray")
axs2[1].set_title('T1 - Coronal View')

axs2[2].imshow(t1_data_sample[:,:,slice_nb], cmap="gray")
axs2[2].set_title('T1 - Axial View')

plt.show()

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize = (15,15))
ax1.imshow(rotate(montage(t1_data_sample[:,:,:]), 90, resize=True), cmap ='gray')

# montage allows us to concatenate multiple images of the same size horizontally and vertically

In [ ]:
# Display all slices of a segmentation
fig, ax1 = plt.subplots(1, 1, figsize = (15,15))
ax1.imshow(rotate(montage(seg_data_sample[:,:,:]), 90, resize=True), cmap ='gray')

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize = (15,15))
ax1.imshow(rotate(montage(t1_data_sample[60:135,:,:]), 90, resize=True), cmap ='gray')

In [ ]:
# Plot a segmantation
some_seg_img = nib.load(f'{brats_sample_path}seg.nii.gz').get_fdata()

cmap = mpl.colors.ListedColormap(['#440054', '#3b528b', '#18b880', '#e6d74f'])
norm = mpl.colors.BoundaryNorm([-0.5, 0.5, 1.5, 2.5, 3.5], cmap.N)

plt.imshow(some_seg_img[100,:,:], cmap=cmap, norm=norm)
plt.title(brats_index)
plt.colorbar()

In [ ]:
seg_samples = [os.path.join(data_path_GLI_train_dir, sample, f"{sample}-seg.nii.gz") for sample in os.listdir(data_path_GLI_train_dir) if not sample.endswith('.csv')]

saved_values = []
max_nb_values = 0
for sample in seg_samples:
    seg_img_sample = nib.load(sample).get_fdata()
    unique_values = np.unique(seg_img_sample)
    nb_unique_values = len(np.unique(seg_img_sample))
    
    if nb_unique_values > max_nb_values:
        max_nb_values = nb_unique_values
        saved_values = unique_values

print(f"Maximum number of values in all segmentation images: {max_nb_values}")
print(f"Values: {saved_values}")

In [ ]:
# Deletion of class 0
seg_0 = some_seg_img.astype(float).copy()
seg_0[seg_0 != 0] = np.nan

# Isolation of class 1
seg_1 = some_seg_img.astype(float).copy()
seg_1[seg_1 != 1] = np.nan

# Isolation of class 2
seg_2 = some_seg_img.astype(float).copy()
seg_2[seg_2 != 2] = np.nan

# Isolation of class 4
seg_3 = some_seg_img.astype(float).copy()
seg_3[seg_3 != 4] = np.nan

# Define legend
class_names = ['class 0', 'class 1', 'class 2', 'class 3']
legend = [plt.Rectangle((0, 0), 1, 1, color=cmap(i), label=class_names[i]) for i in range(len(class_names))]

fig, axs3 = plt.subplots(1, 5, figsize=(15, 15))

axs3[0].imshow(some_seg_img[100,:,:], cmap=cmap, norm=norm)
axs3[0].set_title('Original Segmentation')
axs3[0].legend(handles=legend, loc='upper right')

axs3[1].imshow(seg_0[100,:,:], cmap=cmap, norm=norm)
axs3[1].set_title('[Not Tumor] class 0')

axs3[2].imshow(seg_1[100,:,:], cmap=cmap, norm=norm)
axs3[2].set_title('[NCR] class 1')

axs3[3].imshow(seg_2[100,:,:], cmap=cmap, norm=norm)
axs3[3].set_title('[ED] class 2')

axs3[4].imshow(seg_3[100,:,:], cmap=cmap, norm=norm)
axs3[4].set_title('[ET] class 3')

# Save the figure to a file
plt.savefig('segmentation_classes.png', dpi=300)  # Saves the image as 'segmentation_classes.png'

plt.show()

In [ ]:
values, counts = np.unique(some_seg_img, return_counts=True)
print(f'distribution of 4 classes: {counts}')

In [ ]:
# Modality shape
print(f'modality shape: {t1_data_sample.shape}')

# Segmentation shape
print(f'segmentation shape: {seg_data_sample.shape}')

### Step 2 - Load images and parameters

In [9]:
# Retrieve all samples from path with listdir(). This method lists of all files + directories in the specified directory.
all_datas = os.listdir(data_path_GLI_train_dir)
print("Number of samples:", len(all_datas))

Number of samples: 1251


In [10]:
# Split the dataset into train and validation sets
datas_train, datas_val = train_test_split(all_datas, test_size=0.2, random_state=42)

# Split the train set into the real train set and in a test set 
datas_train, datas_test = train_test_split(datas_train, test_size=0.15, random_state=42)

# Print data distribution (Train: 68%, Test: 12%, Val: 20%)
print(f"Train length: {len(datas_train)}")
print(f"Validation length: {len(datas_val)}")
print(f"Test length: {len(datas_test)}")

Train length: 850
Validation length: 251
Test length: 150


In [11]:
pip show keras tensorflow # type: ignore

Name: keras
Version: 2.7.0
Summary: Deep learning for humans.
Home-page: https://keras.io/
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache 2.0
Location: /home/cbel/Desktop/YIHAN/venv/lib/python3.8/site-packages
Requires: 
Required-by: tensorflow
---
Name: tensorflow
Version: 2.7.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /home/cbel/Desktop/YIHAN/venv/lib/python3.8/site-packages
Requires: h5py, wheel, typing-extensions, google-pasta, gast, flatbuffers, tensorboard, protobuf, grpcio, opt-einsum, libclang, absl-py, tensorflow-io-gcs-filesystem, tensorflow-estimator, six, termcolor, keras-preprocessing, numpy, wrapt, astunparse, keras
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [12]:
import tensorflow as tf
print("===============================")
print(f'tensorflow version: {tf.__version__}')  # Should print 2.7.0
print(f'is built with CUDA: {tf.test.is_built_with_cuda()}')  # Should return True
print(f'GPU details: {tf.config.list_physical_devices("GPU")}')  # Should show GPU details

# Check if TensorFlow can detect a GPU
if tf.config.list_physical_devices('GPU'):
    print("GPU is available.")
else:
    print("GPU is not available.")

print("===============================")

tensorflow version: 2.7.0
is built with CUDA: True
GPU details: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU is available.


### Skip if no need - combine existing pkl

In [ ]:
import pickle
import csv

DATA_1_DIR = 'processed_data_0.05_0.1'
DATA_2_DIR = 'processed_data_0.1_0.25'
COMBINED_DATA_DIR = 'processed_data_0.05_0.25'
SET = 'test'


with open(f'./data/{DATA_1_DIR}/processed_{SET}_data.pkl', 'rb') as f:
    data1 = pickle.load(f)
with open(f'./data/{DATA_2_DIR}/processed_{SET}_data.pkl', 'rb') as f:
    data2 = pickle.load(f)
combined_data = data1.copy()
combined_data.update(data2)
with open(f'./data/{COMBINED_DATA_DIR}/processed_{SET}_data.pkl', 'wb') as f:
    pickle.dump(combined_data, f)


with open(f'./data/{DATA_1_DIR}/{SET}_data_info.csv', 'r') as csv1, open(f'./data/{DATA_2_DIR}/{SET}_data_info.csv', 'r') as csv2:
    reader1 = csv.reader(csv1)
    reader2 = csv.reader(csv2)

    with open(f'./data/{COMBINED_DATA_DIR}/{SET}_data_info.csv', 'w', newline='') as csv_out:
        writer = csv.writer(csv_out)
        for row in reader1:
            writer.writerow(row)
        next(reader2)
        for row in reader2:
            writer.writerow(row)


### Step * Crop Nii files

#### Step * - 1: Set up params & crop method

In [13]:
import pickle
import os
import nibabel as nib
import cv2
import numpy as np
import csv

total_slices_dict = {
    "train": 0,
    "val": 0,
    "test": 0
}

def preprocess_and_save(data_list, save_path, log_path, tumor_ratio_lower_bound, tumor_ratio_upper_bound):
    processed_data = {'X': [], 'y': []}
    slice_info = {}  # Dictionary to store slice information for each NIfTI

    with open(log_path, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['NIfTI ID', 'Slice Index', 'Tumor Ratio', 'Brain Area'])

        total_slices = 0

        for i in data_list:
            data_path = os.path.join(data_path_GLI_train_dir, i, i)
            t1ce_path = data_path + '-t1c.nii.gz'
            flair_path = data_path + '-t2f.nii.gz'
            seg_path = data_path + '-seg.nii.gz'
            
            t1ce = nib.load(t1ce_path).get_fdata()
            flair = nib.load(flair_path).get_fdata()
            seg = nib.load(seg_path).get_fdata()

            slice_info[i] = []  # Initialize list for each NIfTI

            for j in range(VOLUME_SLICES_PLUS):
                slice_seg = seg[:, :, j + VOLUME_START_AT]
                slice_flair = flair[:, :, j + VOLUME_START_AT]
                tumor_area = np.sum(slice_seg > 0)
                brain_area = np.sum(slice_flair > 0)  # Use non-zero pixels in FLAIR as brain area

                # Avoid division by zero
                if brain_area == 0:
                    continue

                tumor_percentage = tumor_area / brain_area

                if tumor_ratio_lower_bound < tumor_percentage <= tumor_ratio_upper_bound:
                    X_slice = np.zeros((IMG_SIZE, IMG_SIZE, N_CHANNELS))
                    X_slice[:, :, 0] = cv2.resize(slice_flair, (IMG_SIZE, IMG_SIZE))
                    X_slice[:, :, 1] = cv2.resize(t1ce[:, :, j + VOLUME_START_AT], (IMG_SIZE, IMG_SIZE))
                    processed_data['X'].append(X_slice)
                    processed_data['y'].append(slice_seg)
                    slice_info[i].append(j + VOLUME_START_AT)  # Record the slice index

                    # Write slice information to CSV file
                    csv_writer.writerow([i, j + VOLUME_START_AT, round(tumor_percentage, 4), brain_area])
                    total_slices += 1

        total_slices_dict[log_path.split('/')[-1].split('_')[0]] = total_slices

    # Save the processed data
    with open(save_path, 'wb') as f:
        pickle.dump(processed_data, f)

def count_rows_in_csv(file_path):
    with open(file_path, mode='r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader, None)
        row_count = sum(1 for row in reader)
    return row_count

In [14]:
import csv
import os

def append_to_csv(file_path, data, header=None):
    file_exists = os.path.isfile(file_path)

    with open(file_path, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        if header and not file_exists:
            writer.writerow(header)
        writer.writerows(data)


#### Step * - 2: Crop

In [15]:
from datetime import datetime
import gc

processed_data_dir_root = f'./data/processed_data_{TUMOR_RATIO_LOWER_BOUND}_{TUMOR_RATIO_UPPER_BOUND}'
processed_data_lists_csv = f'./data/processed_data_lists.csv'
processed_data_lists_header = ['tumor_ratio_lower_bound', 'tumor_ratio_upper_bound', 'train_slices', 'val_slices', 'test_slices', 'generated_timestamp']

print(f'tumor ratio: {TUMOR_RATIO_LOWER_BOUND} ~ {TUMOR_RATIO_UPPER_BOUND}')

if not os.path.exists(processed_data_dir_root):
    os.makedirs(processed_data_dir_root)
    print("dir processed_data created.")

    preprocess_and_save(datas_train, f'{processed_data_dir_root}/processed_train_data.pkl', f'{processed_data_dir_root}/train_data_info.csv', TUMOR_RATIO_LOWER_BOUND, TUMOR_RATIO_UPPER_BOUND)
    gc.collect()
    preprocess_and_save(datas_val, f'{processed_data_dir_root}/processed_val_data.pkl', f'{processed_data_dir_root}/val_data_info.csv', TUMOR_RATIO_LOWER_BOUND, TUMOR_RATIO_UPPER_BOUND)
    gc.collect()
    preprocess_and_save(datas_test, f'{processed_data_dir_root}/processed_test_data.pkl', f'{processed_data_dir_root}/test_data_info.csv', TUMOR_RATIO_LOWER_BOUND, TUMOR_RATIO_UPPER_BOUND)
    gc.collect()

    processed_data_lists_data = [
        [TUMOR_RATIO_LOWER_BOUND, TUMOR_RATIO_UPPER_BOUND, total_slices_dict["train"], total_slices_dict["val"], total_slices_dict["test"], datetime.now().strftime("%b-%d_%H-%M-%S")]
    ]
    append_to_csv(processed_data_lists_csv, processed_data_lists_data, processed_data_lists_header)

else:
    print(f'dir processed_data existed at {processed_data_dir_root}.')

# set the data info to params
TRAIN_PKL = f'{processed_data_dir_root}/processed_train_data.pkl'
VAL_PKL = f'{processed_data_dir_root}/processed_val_data.pkl'
TEST_PKL = f'{processed_data_dir_root}/processed_test_data.pkl'
TRAIN_PKL_CSV = f'{processed_data_dir_root}/train_data_info.csv'
VAL_PKL_CSV = f'{processed_data_dir_root}/val_data_info.csv'
TEST_PKL_CSV = f'{processed_data_dir_root}/test_data_info.csv'
total_slices_dict.update({
    "train": count_rows_in_csv(TRAIN_PKL_CSV),
    "val": count_rows_in_csv(VAL_PKL_CSV),
    "test": count_rows_in_csv(TEST_PKL_CSV)
})

print(f'train: {total_slices_dict["train"]} slices')
print(f'val: {total_slices_dict["val"]} slices')
print(f'test: {total_slices_dict["test"]} slices')

tumor ratio: 0.1 ~ 0.15
dir processed_data created.
train: 9830 slices
val: 2862 slices
test: 2118 slices


### Step 3 - Set up training compoenents

**When generating `DataGenerator`**:
- We use a data generator to be able to process and send our data to our neural network (since all our images cannot be stored in memory at once).
- For each epoch (single pass of the entire training dataset through a neural network), the model will receive 250 samples (those contained in our training dataset).
- For each sample, the model will have to analyze 150 slices (since there are two modalities, and 75(VOLUME_SLICES) selected slices for both of them), received in a (128, 128) shape, as an X array of a (128, 128, 75, 2) shape. This array will be provided with the ground truth segmentation of the patient, which will be One-Hot encoded and will then have a (75, 128, 128, 4) shape.

#### 3 - 1: DataGenerator

##### 3-1-1 DataGenerator_UnetV1

In [16]:
from tensorflow.keras import layers, models

# DataGenerator from Rastislav's notebook, https://www.kaggle.com/code/rastislav/3d-mri-brain-tumor-segmentation-u-net

class DataGenerator_UnetV1(keras.utils.all_utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, dim=(IMG_SIZE,IMG_SIZE), batch_size = 1, n_channels = N_CHANNELS, shuffle=True):
        print('------ usging Data Generator V1 ------')
        'Initialization'
        self.dim = dim # Resized image dimensions (128 x 128)
        self.batch_size = batch_size #  Number of images to load each time
        self.list_IDs = list_IDs # Patients IDs
        self.n_channels = n_channels # Number of channels (T1CE + FLAIR)
        self.shuffle = shuffle # Indicates if data is shuffled for each epoch
        self.on_epoch_end() # Updates indexes after each epoch

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        Batch_ids = [self.list_IDs[k] for k in indexes]

        # Load & Generate data
        X, y = self.__data_generation(Batch_ids)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, Batch_ids):
        'Generates data containing batch_size samples'
        # Initialization
        X = np.zeros((self.batch_size*VOLUME_SLICES_PLUS, *self.dim, self.n_channels))
        y = np.zeros((self.batch_size*VOLUME_SLICES_PLUS, 240, 240))

        # Generate data
        for c, i in enumerate(Batch_ids):
            
            # Get path of each RMI modality and the segmentation
            data_path = os.path.join(data_path_GLI_train_dir, i, i)
            t1ce_path = data_path + '-t1c.nii.gz'
            flair_path = data_path + '-t2f.nii.gz'
            seg_path = data_path + '-seg.nii.gz'
            #t1_path = sample_path + '_t1.nii.gz'
            #t2_path = sample_path + '_t2.nii.gz'
            
            # Extract the data from these paths
            t1ce = nib.load(t1ce_path).get_fdata()
            flair = nib.load(flair_path).get_fdata()
            seg = nib.load(seg_path).get_fdata()
            #t1 = nib.load(t1_paths).get_fdata()
            #t2 = nib.load(t2_path).get_fdata()
        
            for j in range(VOLUME_SLICES_PLUS):
                 X[j +VOLUME_SLICES_PLUS*c,:,:,0] = cv2.resize(flair[:,:,j+VOLUME_START_AT], (IMG_SIZE, IMG_SIZE))
                 X[j +VOLUME_SLICES_PLUS*c,:,:,1] = cv2.resize(t1ce[:,:,j+VOLUME_START_AT], (IMG_SIZE, IMG_SIZE))

                 y[j +VOLUME_SLICES_PLUS*c] = seg[:,:,j+VOLUME_START_AT]
                    
        # Masks / Segmentations
        y[y==4] = 3
        mask = tensorflow.one_hot(y, 4)
        Y = tensorflow.image.resize(mask, (IMG_SIZE, IMG_SIZE))
        
        # Scale data between 0 and 1 (since the minimum value in the data is 0)
        return X/np.max(X), Y

##### 3-1-2 DataGenerator_UnetV2

In [17]:
from tensorflow.keras import layers, models

# DataGenerator from Rastislav's notebook, https://www.kaggle.com/code/rastislav/3d-mri-brain-tumor-segmentation-u-net

class DataGenerator_UnetV2(keras.utils.all_utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, dim=(IMG_SIZE,IMG_SIZE), batch_size = 1, n_channels = N_CHANNELS, shuffle=True):
        print('------ usging Data Generator V2 ------')
        'Initialization'
        self.dim = dim # Resized image dimensions (128 x 128)
        self.batch_size = batch_size #  Number of images to load each time
        self.list_IDs = list_IDs # Patients IDs
        self.n_channels = n_channels # Number of channels (T1CE + FLAIR)
        self.shuffle = shuffle # Indicates if data is shuffled for each epoch
        self.on_epoch_end() # Updates indexes after each epoch

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        Batch_ids = [self.list_IDs[k] for k in indexes]

        # Load & Generate data
        X, y = self.__data_generation(Batch_ids)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, Batch_ids):
        'Generates data containing batch_size samples'
        # Initialization
        X = np.zeros((self.batch_size*VOLUME_SLICES_PLUS, *self.dim, self.n_channels))
        y = np.zeros((self.batch_size*VOLUME_SLICES_PLUS, 240, 240))

        # Generate data
        for c, i in enumerate(Batch_ids):
            
            # Get path of each RMI modality and the segmentation
            data_path = os.path.join(data_path_GLI_train_dir, i, i)
            t1ce_path = data_path + '-t1c.nii.gz'
            flair_path = data_path + '-t2f.nii.gz'
            seg_path = data_path + '-seg.nii.gz'
            #t1_path = sample_path + '_t1.nii.gz'
            #t2_path = sample_path + '_t2.nii.gz'
            
            # Extract the data from these paths
            t1ce = nib.load(t1ce_path).get_fdata()
            flair = nib.load(flair_path).get_fdata()
            seg = nib.load(seg_path).get_fdata()
            #t1 = nib.load(t1_paths).get_fdata()
            #t2 = nib.load(t2_path).get_fdata()
        
            for j in range(VOLUME_SLICES_PLUS):
                 X[j +VOLUME_SLICES_PLUS*c,:,:,0] = cv2.resize(flair[:,:,j+VOLUME_START_AT], (IMG_SIZE, IMG_SIZE))

                 y[j +VOLUME_SLICES_PLUS*c] = seg[:,:,j+VOLUME_START_AT]
                    
        # Masks / Segmentations
        y[y==4] = 3
        mask = tensorflow.one_hot(y, 4)
        Y = tensorflow.image.resize(mask, (IMG_SIZE, IMG_SIZE))
        
        # Scale data between 0 and 1 (since the minimum value in the data is 0)
        return X/np.max(X), Y

##### 3-1-3 DataGenerator_UnetV3_count_image_ratio

In [18]:
from tensorflow.keras import layers, models

class DataGenerator_UnetV3_count_image_ratio(keras.utils.all_utils.Sequence):
    def __init__(self, data_path, batch_size=1, shuffle=True):
        print('------ usging Data Generator V3: use crop NIIs ------')
        with open(data_path, 'rb') as f:
            self.data = pickle.load(f)
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.data['X']))
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.data['X']) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        X_batch = [self.data['X'][k] for k in indexes]
        y_batch = [self.data['y'][k] for k in indexes]

        y_batch = np.array(y_batch)
        y_batch[y_batch == 4] = 3
        mask = tensorflow.one_hot(y_batch, 4)
        Y = tensorflow.image.resize(mask, (IMG_SIZE, IMG_SIZE))

        return np.array(X_batch) / np.max(X_batch), Y

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

#### 3 -2: U-Net model

##### 3-2-1 U-Net v1

In [19]:
# U-Net implementation for BraTS 2019 by Naomi Fridman, https://naomi-fridman.medium.com/multi-class-image-segmentation-a5cc671e647a
def unet_v1(inputs, ker_init, dropout):
    print('****** using U-Net V1 ******')
    conv1 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(inputs)
    conv1 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(conv1)
    
    pool = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(pool)
    conv = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(conv)
    
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(conv2)
    
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(conv3)
    
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv5 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(pool4)
    conv5 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(conv5)
    drop5 = Dropout(dropout)(conv5)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(UpSampling2D(size = 2)(drop5))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(UpSampling2D(size = 2)(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(UpSampling2D(size = 2)(conv8))
    merge9 = concatenate([conv,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(conv9)
    
    up = Conv2D(32, 2, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(UpSampling2D(size = 2)(conv9))
    merge = concatenate([conv1,up], axis = 3)
    conv = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(merge)
    conv = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(conv)
    
    conv10 = Conv2D(4, 1, activation = 'softmax')(conv)
    
    return Model(inputs = inputs, outputs = conv10)

##### 3-2-2: U-Net V2

In [20]:
def unet_v2(inputs, ker_init, dropout=0.5):
    print('****** using U-Net V2 ******')
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(conv1)

    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(conv2)
    
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(conv3)
    
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(conv4)
    drop4 = Dropout(dropout)(conv4)
    
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(conv5)
    drop5 = Dropout(dropout)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = ker_init)(conv9)

    conv10 = Conv2D(4, 1, activation = 'sigmoid')(conv9)

    return Model(inputs = inputs, outputs = conv10)

#### 3 - 3: Metrics

In [21]:
# Compute metric between the predicted segmentation and the ground truth
def dice_coef(y_true, y_pred, smooth=1.0):
    class_num = 4
    for i in range(class_num):
        y_true_f = K.flatten(y_true[:,:,:,i])
        y_pred_f = K.flatten(y_pred[:,:,:,i])
        intersection = K.sum(y_true_f * y_pred_f)
        loss = ((2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth))
        if i == 0:
            total_loss = loss
        else:
            total_loss = total_loss + loss
    total_loss = total_loss / class_num
    return total_loss

def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())


def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

########################### my customized metrics ###########################

def sensibility(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    false_positives = K.sum(K.round(K.clip(y_pred - y_true, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    # Calculate the sensibility
    sensibility = 1 - (false_positives / (true_positives + possible_positives + K.epsilon()))
    # Check for NaN or Inf values and return NaN if True
    sensibility = tf.where(tf.math.is_nan(sensibility) | tf.math.is_inf(sensibility),
                           tf.constant(float('NaN')),
                           sensibility)
    return sensibility


def jaccard(y_true, y_pred):
    # True positives, false positives, false negatives
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    false_positives = K.sum(K.round(K.clip(y_pred - y_true, 0, 1)))
    false_negatives = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)))
    # Calculate Jaccard index
    denominator = true_positives + false_positives + false_negatives + K.epsilon()  # Add epsilon to avoid division by zero
    result = true_positives / denominator
    # Handle NaN or Inf cases
    result = K.switch(K.is_inf(result) | K.is_nan(result), K.constant(float('NaN')), result)
    return result


# Hausdorff distance (approximation for use in Keras)
def hausdorff(y_true, y_pred):
    # Flatten the coordinates of the true and predicted segmentation masks
    y_true_flat = K.flatten(y_true)
    y_pred_flat = K.flatten(y_pred)
    # Compute the Euclidean distance between every pair of points
    dist_matrix = K.square(y_true_flat - y_pred_flat)
    # Get the maximum distance in both directions (directed Hausdorff)
    directed_hausdorff_1 = K.max(dist_matrix, axis=1)  # From y_true to y_pred
    directed_hausdorff_2 = K.max(dist_matrix, axis=0)  # From y_pred to y_true
    # Take the maximum of both directed Hausdorff distances
    hausdorff_dist = K.maximum(directed_hausdorff_1, directed_hausdorff_2)
    # Handle NaN or Inf values by replacing them with NaN (if needed)
    hausdorff_dist = K.switch(K.is_inf(hausdorff_dist) | K.is_nan(hausdorff_dist), K.constant(float('NaN')), hausdorff_dist)
    return hausdorff_dist

#### 3 - 4: Define training params

In [22]:
from keras import Input

MODEL_VERSION = 1
DATA_GENERATOR_VERSION = 3

EPOCHS = 200
STEPS_PER_EPOCH = len(datas_train)

INPUT_LAYER = Input((IMG_SIZE, IMG_SIZE, N_CHANNELS))
KER_INIT = 'he_normal'
DROPOUT = 0.2
LEARNING_RATE = 1e-4
LOSS_FUNCTION = "categorical_crossentropy"

#### 3 - 5: Build the model

In [23]:
# Build and compile the model
model_builders = {
    1: unet_v1,
    2: unet_v2
}
model = model_builders.get(MODEL_VERSION, unet_v1)(inputs=INPUT_LAYER, ker_init=KER_INIT, dropout=DROPOUT)

model.compile(loss=LOSS_FUNCTION, optimizer=tensorflow.keras.optimizers.Adam(learning_rate=LEARNING_RATE), metrics = ['accuracy',tensorflow.keras.metrics.MeanIoU(num_classes=4), dice_coef, precision, sensitivity, specificity ] )

****** using U-Net V1 ******


#### 3 - 6: Set up DataGenerator

In [24]:
data_generators = {
    1: DataGenerator_UnetV1,
    2: DataGenerator_UnetV2,
    3: DataGenerator_UnetV3_count_image_ratio
}

training_generator = data_generators.get(DATA_GENERATOR_VERSION, DataGenerator_UnetV1)(TRAIN_PKL if DATA_GENERATOR_VERSION == 3 else datas_train)
valid_generator = data_generators.get(DATA_GENERATOR_VERSION, DataGenerator_UnetV1)(VAL_PKL if DATA_GENERATOR_VERSION == 3 else datas_val)

------ usging Data Generator V3: use crop NIIs ------
------ usging Data Generator V3: use crop NIIs ------


#### 3 - 7: Do the GC and check the GPU and memory status

In [25]:
import tensorflow as tf
import gc

gc.collect()

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for i, gpu in enumerate(gpus):
        device_name = f"GPU:{i}"  # Properly format the device name
        details = tf.config.experimental.get_memory_info(device_name)
        print(f"GPU: {device_name}")
        print(f"Current Memory Usage: {details['current']} bytes")
        print(f"Peak Memory Usage: {details['peak']} bytes")
else:
    print("No GPU devices found.")

for gpu in gpus:
    print(f"Memory growth for {gpu}: {tf.config.experimental.get_memory_growth(gpu)}")

GPU: GPU:0
Current Memory Usage: 31040804 bytes
Peak Memory Usage: 37721124 bytes
Memory growth for PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'): None


In [26]:
# Mixed Precision Training
# from tensorflow.keras.mixed_precision import experimental as mixed_precision

# policy = mixed_precision.Policy('mixed_float16')
# mixed_precision.set_policy(policy)

#### 3 - 8: Set up callbacks

Callbacks are functions that can be executed during the training process. 

We will use three callbacks:

- **ReduceLROnPlateau**: This callback reduces the learning rate when a metric has stopped improving (validation loss here). The learning rate is reduced by a factor of 0.2, the patience is set to 2 epochs, and the minimum learning rate is set to 0.000001.

- **ModelCheckpoint**: Saves the best model weights (model that has obtained the lowest validation loss during the different epochs). Saving a model allows us to reuse it later or to share it, without having to retrain it from scratch. This will save us time and resources!

- **CSVLogger**: Add metrics to a CSV file, which is named *training.log* (parameter `append` is set to `False` so the file is overwritten if it already exists). 

In [27]:
from datetime import datetime
import os

# Generate timestamp for filenames
start_train_timestamp = datetime.now().strftime("%b-%d_%H-%M-%S")

model_dir = f'./model/model_v{MODEL_VERSION}_{start_train_timestamp}'


if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    print(f"Model dir {model_dir} created.")
else:
    print(f"Model dir {model_dir} existed.")

# Define file paths with timestamps
training_log_filename = f"{model_dir}/training-{start_train_timestamp}.log"
model_checkpoint_path = f"{model_dir}/model_epoch{{epoch:02d}}-val_loss{{val_loss:.6f}}_{start_train_timestamp}.m5"

callbacks = [
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.000001, verbose=1),
    
    keras.callbacks.ModelCheckpoint(filepath=model_checkpoint_path, verbose=1, save_best_only=True, save_weights_only=True),

    CSVLogger(training_log_filename, separator=',', append=False)  # Save logs with timestamp
]

Model dir ./model/model_v1_Mar-22_15-37-10 created.


### Step 4 - Train & Save the model

#### 4 - 1: log all cell outputs during training

In [28]:
import sys

print(f'----------')
print(f'model name: {model_builders[MODEL_VERSION].__name__}')
print(f'epochs: {EPOCHS}')
print(f'steps_per_epoch: {STEPS_PER_EPOCH}')
print(f'dimension: {IMG_SIZE}')
print(f'n_channels: {N_CHANNELS}')
print(f'input layer: {INPUT_LAYER}')
print(f'kernel init: {KER_INIT}')
print(f'dropout: {DROPOUT}')
print(f'learning rate: {LEARNING_RATE}')
print(f'loss function: {LOSS_FUNCTION}')
    
print(f"train cases: {len(datas_train)}")
print(f'train slices: {total_slices_dict["train"]}')
print(f"validation cases: {len(datas_val)}")
print(f'val slices: {total_slices_dict["val"]}')
print(f'slice index: {VOLUME_START_AT} ~ {VOLUME_START_AT + VOLUME_SLICES_PLUS}')
print(f'tumor ratio: {TUMOR_RATIO_LOWER_BOUND} ~ {TUMOR_RATIO_UPPER_BOUND}')
print(f"test cases: {len(datas_test)}")

print(f'start training at: {start_train_timestamp}')
print(f'----------')

original_stdout = sys.stdout
with open(f'{model_dir}/info.log', 'w') as predict_log_file:
    sys.stdout = predict_log_file
    print(f'----------')
    print(f'model name: {model_builders[MODEL_VERSION].__name__}')
    print(f'epochs: {EPOCHS}')
    print(f'steps_per_epoch: {STEPS_PER_EPOCH}')
    print(f'dimension: {IMG_SIZE}')
    print(f'n_channels: {N_CHANNELS}')
    print(f'input layer: {INPUT_LAYER}')
    print(f'kernel init: {KER_INIT}')
    print(f'dropout: {DROPOUT}')
    print(f'learning rate: {LEARNING_RATE}')
    print(f'loss function: {LOSS_FUNCTION}')

    print(f"train cases: {len(datas_train)}")
    print(f'train slices: {total_slices_dict["train"]}')
    print(f"validation cases: {len(datas_val)}")
    print(f'val slices: {total_slices_dict["val"]}')
    print(f'slice index: {VOLUME_START_AT} ~ {VOLUME_START_AT + VOLUME_SLICES_PLUS}')
    print(f'tumor ratio: {TUMOR_RATIO_LOWER_BOUND} ~ {TUMOR_RATIO_UPPER_BOUND}')
    print(f"test cases: {len(datas_test)}")

    print(f'start training at: {start_train_timestamp}')
    print(f'----------')

sys.stdout = original_stdout

----------
model name: unet_v1
epochs: 200
steps_per_epoch: 850
dimension: 256
n_channels: 2
input layer: KerasTensor(type_spec=TensorSpec(shape=(None, 256, 256, 2), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")
kernel init: he_normal
dropout: 0.2
learning rate: 0.0001
loss function: categorical_crossentropy
train cases: 850
train slices: 9830
validation cases: 251
val slices: 2862
slice index: 40 ~ 116
tumor ratio: 0.1 ~ 0.15
test cases: 150
start training at: Mar-22_15-37-10
----------


#### 4 - 2: train

In [ ]:
model.fit(training_generator,
          epochs=EPOCHS,
          steps_per_epoch=STEPS_PER_EPOCH,
          callbacks=callbacks,
          validation_data=valid_generator)

Epoch 1/200


2025-03-22 15:37:12.639494: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202


850/850 [==============================] - ETA: 0s - loss: 0.1222 - accuracy: 0.9645 - mean_io_u: 0.6447 - dice_coef: 0.3456 - precision: 0.9696 - sensitivity: 0.9502 - specificity: 0.9929
Epoch 00001: val_loss improved from inf to 0.07296, saving model to ./model/model_v1_Mar-22_15-37-10/model_epoch01-val_loss0.072959_Mar-22_15-37-10.m5
850/850 [==============================] - 29s 29ms/step - loss: 0.1222 - accuracy: 0.9645 - mean_io_u: 0.6447 - dice_coef: 0.3456 - precision: 0.9696 - sensitivity: 0.9502 - specificity: 0.9929 - val_loss: 0.0730 - val_accuracy: 0.9770 - val_mean_io_u: 0.7659 - val_dice_coef: 0.3943 - val_precision: 0.9851 - val_sensitivity: 0.9669 - val_specificity: 0.9948 - lr: 1.0000e-04
Epoch 2/200
849/850 [============================>.] - ETA: 0s - loss: 0.0648 - accuracy: 0.9788 - mean_io_u: 0.7212 - dice_coef: 0.4327 - precision: 0.9854 - sensitivity: 0.9702 - specificity: 0.9949

#### 4 - 3: Plot the training process

In [ ]:
import glob

training_log_filename = glob.glob(os.path.join(model_dir, 'training-*.log'))[0]
plt_filename = os.path.splitext(os.path.basename(training_log_filename))[0]
print(f'plt image will be saved in: {training_log_filename}\nwith name: {plt_filename}')

In [ ]:
# Read the CSVlogger file that contains all our metrics (accuracy, loss, dice_coef, ...) of our training
history = pd.read_csv(training_log_filename, sep=',', engine='python')

# Plot training and validation metrics
fig, axs = plt.subplots(1, 3, figsize=(16, 8))
# fig, axs = plt.subplots(1, 4, figsize=(16, 8))

axs[0].plot(history['epoch'], history['accuracy'], 'b', label='Training Accuracy')
axs[0].plot(history['epoch'], history['val_accuracy'], 'r', label='Validation Accuracy')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Accuracy')
axs[0].legend()

axs[1].plot(history['epoch'], history['loss'], 'b', label='Training Loss')
axs[1].plot(history['epoch'], history['val_loss'], 'r', label='Validation Loss')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Loss')
axs[1].legend()

axs[2].plot(history['epoch'], history['dice_coef'], 'b', label='Training dice coef')
axs[2].plot(history['epoch'], history['val_dice_coef'], 'r', label='Validation dice coef')
axs[2].set_xlabel('Epoch')
axs[2].set_ylabel('Dice Coefficient')
axs[2].legend()

# axs[3].plot(history['epoch'], history['mean_io_u'], 'b', label='Training mean IOU')
# axs[3].plot(history['epoch'], history['val_mean_io_u'], 'r', label='Validation mean IOU')
# axs[3].set_xlabel('Epoch')
# axs[3].set_ylabel('Mean IOU')
# axs[3].legend()

# Add space between subplots
plt.subplots_adjust(wspace=0.4)
plt.savefig(f'./plt/train/{plt_filename}.png', dpi=300)

plt.show()

### Step 5 - Predict

In [ ]:
import glob
import os

def find_latest_file(directory, pattern):
    search_pattern = os.path.join(directory, pattern)
    files = glob.glob(search_pattern)

    if not files:
        return None

    latest_file = max(files, key=os.path.getmtime)

    return latest_file

PATTERN = 'model_epoch*-val_loss*_*.m5.index'

latest_model_file = find_latest_file(f'{model_dir}', PATTERN)

latest_trained_model = os.path.splitext(os.path.splitext(os.path.basename(latest_model_file))[0])[0]

print(f'latest model file: {latest_trained_model}')

In [ ]:
# Compile a model and load our saved weights
from tensorflow.keras import Input

INPUT_LAYER = Input((IMG_SIZE, IMG_SIZE, N_CHANNELS))
KER_INIT = KER_INIT
DROPOUT = DROPOUT
LEARNING_RATE = LEARNING_RATE

best_saved_model = model_builders.get(MODEL_VERSION, unet_v1)(inputs=INPUT_LAYER, ker_init=KER_INIT, dropout=DROPOUT)

best_saved_model.compile(loss="categorical_crossentropy", optimizer=tensorflow.keras.optimizers.Adam(learning_rate=LEARNING_RATE), metrics = ['accuracy',tensorflow.keras.metrics.MeanIoU(num_classes=4), dice_coef, precision, sensitivity, specificity] )


best_saved_model.load_weights(f'{model_dir}/{latest_trained_model}.m5')

In [ ]:
def predict_segmentation(sample_path):
    # Load NIfTI (.nii) files of the sample (patient)
    t1ce_path = sample_path + '-t1c.nii.gz'
    flair_path = sample_path + '-t2f.nii.gz'
    #t1_path = sample_path + '_t1.nii'
    #t2_path = sample_path + '_t2.nii'
            
    # Extract the data from these paths
    t1ce = nib.load(t1ce_path).get_fdata()
    flair = nib.load(flair_path).get_fdata()
    
    # Create an empty array
    X = np.empty((VOLUME_SLICES_PLUS, IMG_SIZE, IMG_SIZE, 2))
    
    # Perform the same operations as our DataGenerator, to keep the same input shape
    for j in range(VOLUME_SLICES_PLUS):
        X[j,:,:,0] = cv2.resize(flair[:,:,j+VOLUME_START_AT], (IMG_SIZE,IMG_SIZE))
        X[j,:,:,1] = cv2.resize(t1ce[:,:,j+VOLUME_START_AT], (IMG_SIZE,IMG_SIZE))
        
    # Send our images to the CNN model and return predicted segmentation 
    return model.predict(X/np.max(X), verbose=1)

In [ ]:
def show_predicted_segmentations(samples_list, slice_to_plot, cmap, norm):
    # Choose a random patient
    random_sample = random.choice(samples_list)
    print(f'random_sample: {random_sample}')
    
    # Get path of this patient
    random_sample_path = os.path.join(data_path_GLI_train_dir, random_sample, random_sample)
    print(f'random_sample_path: {random_sample_path}')
    
    # Predict patient's segmentation
    predicted_seg = predict_segmentation(random_sample_path)
   
    # Load patient's original segmentation (Ground truth)
    seg_path = random_sample_path + '-seg.nii.gz'
    seg = nib.load(seg_path).get_fdata()
    
    # Resize original segmentation to the same dimensions of the predictions. (Add VOLUME_START_AT because original segmentation contains 155 slices vs only 75 for our prediction)
    seg=cv2.resize(seg[:,:,slice_to_plot+VOLUME_START_AT], (IMG_SIZE, IMG_SIZE), interpolation = cv2.INTER_NEAREST)
    
    # Differentiate segmentations by their labels
    all = predicted_seg[slice_to_plot,:,:,1:4] # Deletion of class 0 (Keep only Core + Edema + Enhancing classes)
    zero = predicted_seg[slice_to_plot,:,:,0] # Isolation of class 0, Background (kind of useless, it is the opposite of the "all")
    first = predicted_seg[slice_to_plot,:,:,1] # Isolation of class 1, Core
    second = predicted_seg[slice_to_plot,:,:,2] # Isolation of class 2, Edema
    third = predicted_seg[slice_to_plot,:,:,3] # Isolation of class 3, Enhancing

    convert_all = all.astype(np.float32) if all.dtype == np.float16 else all
    convert_zero = zero.astype(np.float32) if zero.dtype == np.float16 else zero
    convert_first = first.astype(np.float32) if first.dtype == np.float16 else first
    convert_second = second.astype(np.float32) if second.dtype == np.float16 else second
    convert_third = third.astype(np.float32) if third.dtype == np.float16 else third

    # Plot Original segmentation & predicted segmentation
    print("Patient number: ", random_sample)
    fig, axstest = plt.subplots(1, 6, figsize=(25, 20))

    # Original segmentation
    axstest[0].imshow(seg, cmap, norm)
    axstest[0].set_title('Original Segmentation')
    
    # Layers 1, 2, 3
    axstest[1].imshow(convert_all)
    axstest[1].set_title('Predicted Segmentation - all layers')
    
    # Layer 0
    axstest[2].imshow(convert_zero)
    axstest[2].set_title('Predicted Segmentation - layer 0')
    
    # Layer 1
    axstest[3].imshow(convert_first)
    axstest[3].set_title('Predicted Segmentation - layer 1')
    
    # Layer 2
    axstest[4].imshow(convert_second)
    axstest[4].set_title('Predicted Segmentation - layer 2')
    
    # Layer 3
    axstest[5].imshow(convert_third)
    axstest[5].set_title('Predicted Segmentation - layer 3')
    
    # Add space between subplots
    plt.subplots_adjust(wspace=0.8)

    plt.savefig(f'./plt/predict/{model_dir.split('./model/')[1]}/{random_sample}.png', dpi=300)

    plt.show()

cmap = mpl.colors.ListedColormap(['#440054', '#3b528b', '#18b880', '#e6d74f'])
norm = mpl.colors.BoundaryNorm([-0.5, 0.5, 1.5, 2.5, 3.5], cmap.N)

In [ ]:
show_predicted_segmentations(datas_test, 60, cmap, norm)

In [ ]:
show_predicted_segmentations(datas_test, 50, cmap, norm)

In [ ]:
show_predicted_segmentations(datas_test, 70, cmap, norm)

In [ ]:
def show_post_processed_segmentations(sample, slice_to_plot, cmap, norm):
    
    # Get path of this patient
    sample_path = os.path.join(data_path_GLI_train_dir, sample, sample)
    
    # Predict patient's segmentation
    predicted_seg = predict_segmentation(sample_path)
   
    # Load patient's original segmentation (Ground truth)
    seg_path = sample_path + '-seg.nii.gz'
    seg = nib.load(seg_path).get_fdata()
    
    # Resize original segmentation to the same dimensions of the predictions. (Add VOLUME_START_AT because original segmentation contains 155 slices vs only 75 for our prediction)
    seg=cv2.resize(seg[:,:,slice_to_plot+VOLUME_START_AT], (IMG_SIZE, IMG_SIZE), interpolation = cv2.INTER_NEAREST)
    
    # Fix 4 to 3 to have the same values as in the predicted segmentation, and then same colors
    seg[seg==4] = 3
    
    # Remove background layer (0) from original segmentation
    seg[seg==0] = np.nan
    
    # Post-processing
    # Get indexes for each class of the highest probability pixels. Array will then contain only [0 1 2 3] instead of probabilities
    my_pred = np.argmax(predicted_seg, axis=3)
    my_pred = my_pred[slice_to_plot, :, :]

    # Remove background layer (0) from post-processed predicted segmentation
    # To fix 0 to np.nan, we need to convert array as a float
    my_pred = my_pred.astype(float)
    my_pred[my_pred == 0] = np.nan

    # Remove background layer (0) from classical predicted segmentation
    all = predicted_seg[slice_to_plot,:,:,1:4] 

    convert_all = all.astype(np.float32) if all.dtype == np.float16 else all
    
    # Plot Original segmentation & predicted segmentation without processing & predicted segmentation
    print("Patient number: ", sample)
    fig, axstest = plt.subplots(1, 3, figsize=(15, 10))

    axstest[0].imshow(seg, cmap, norm)
    axstest[0].set_title('Original Segmentation')
    
    axstest[1].imshow(convert_all)
    axstest[1].set_title('Prediction (w/o post processing (layer 1,2,3)')
    
    axstest[2].imshow(my_pred, cmap, norm)
    axstest[2].set_title('Prediction (w/ post processing (layer 1,2,3)')
    
    # Add space between subplots
    plt.subplots_adjust(wspace=0.8)
    plt.savefig(f'./plt/predict/{model_dir.split('./model/')[1]}/{sample}.png', dpi=300)
    
    plt.show()

In [ ]:
show_post_processed_segmentations(sample = "BraTS-GLI-00777-000", slice_to_plot=60, cmap=cmap, norm=norm)

In [ ]:
show_post_processed_segmentations(sample = "BraTS-GLI-00675-000", slice_to_plot=50, cmap=cmap, norm=norm)

### Step 6 - Evaluate the model

In [ ]:
import sys
# Evaluate the model on the test data
test_generator = data_generators.get(DATA_GENERATOR_VERSION, DataGenerator_UnetV1)(TEST_PKL if DATA_GENERATOR_VERSION == 3 else datas_test)
results = model.evaluate(test_generator, batch_size=200, callbacks= callbacks)

descriptions = ["Loss", "Accuracy", "MeanIOU", "Dice coefficient", "Precision", "Sensitivity", "Specificity"]

# Combine results list and descriptions list
results_list = zip(results, descriptions)

print("\nModel evaluation on the test set:")

original_stdout = sys.stdout
with open(f'{model_dir}/predict_result.log', 'w') as predict_log_file:
    sys.stdout = predict_log_file

    # Display each metric with its description
    print("==================================")
    for i, (metric, description) in enumerate(results_list):
        print(f"{description} : {round(metric, 4)}")
    print("==================================")

sys.stdout = original_stdout